# Score Function to optimize the path of UUV
provide score fields for different cases ('consideration') for UUV sim

In [ ]:
import xarray as xr
import numpy as np

## Score function to provide score for each case
INPUT: [lon, lat, depth, timestep]
OUTPUT: [score1, score2, score3]

1. O2 gradient: stronger gradient has higher score
2. OUR: larger magnitude has higher score, need O2, water age, temperature, and salinity
3. Current: ?

In [ ]:
def get_o2_age(lon, lat, depth, method='nearest'):
    " get Oxygen [mmol/(m^3 s)] and water age from OCIM at a giving location"

    OCIM_nc_file = 'OCIM_model_output.nc'
    ds = xr.open_dataset(OCIM_nc)

    point = ds.sel(lon=lon, lat=lat, depth=depth, method=method)
    
    O2 = float(point['O2'].values)
    age = float(point['age'].values)
    
    return O2, age

In [ ]:
# Example usage
robot_location = (123.45, 37.78, 10.0)  # (lon, lat, depth)
O2, age = get_o2_age(*robot_location)

print("O2:", O2)
print("Age:", age)